In [ ]:
#| default_exp model

In [7]:
#| export
import clip
import torch
import tempfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
#from projetMAB1.storage import Storage

In [8]:
#| export
class Model:

    _instance = None

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls, *args, **kwargs)
        return cls._instance
    
    def __init__(self) :
        self._model, self._preprocess = clip.load("ViT-B/32")
        self._model.cpu().eval()
        self._df = pd.read_pickle("../data/runtime/index.pkl")

    def get_image_features(self, images):
        torch_images = [self._preprocess(Image.fromarray(image)) for image in images]
        batch = torch.tensor(np.stack(torch_images)).cpu()

        with torch.no_grad():
            image_features = self._model.encode_image(batch)

        image_features /= image_features.norm(dim = -1, keepdim = True)
        return image_features
    
    def query(self, text:str, k=3):

        images = [Image.open("../data/runtime/frames/" + file_name) for file_name in self._df["preview_name"]]
        images_prep = [self._preprocess(image) for image in images]
        image_input = torch.tensor(np.stack(images_prep)).cpu()
        text_tokens = clip.tokenize(["This is " + desc for desc in [text]]).cpu()

        with torch.no_grad():
            image_features = self._model.encode_image(image_input).float()
            text_features = self._model.encode_text(text_tokens).float()

        image_features /= image_features.norm(dim = -1, keepdim = True)
        text_features /= text_features.norm(dim = -1, keepdim = True)
        similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

        indices = np.flip(np.argpartition(similarity[0], -k)[-k:])
        preview_names = self._df["preview_name"].iloc[indices]

        matches = []
        
        for i, preview_name in zip(indices, preview_names):
            matches.append(("../data/runtime/frames/" + preview_name, similarity[0][i]))

        return matches

In [9]:
pd.read_pickle("../data/runtime/index.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '../data/runtime/index.pkl'

In [ ]:
model = Model()

In [ ]:
matches = model.query("anmie girl")
matches[0][0]

In [ ]:
for match in matches:
    print(match)

In [ ]:
matches = model.query("man playing guitar")
matches[0][0]

In [ ]:
matches = model.query("an anime with a man on a stick")
matches[0][0]

In [ ]:
df = pd.read_pickle("../data/runtime/index.pkl")
df

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()